In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf_v1
from sklearn.model_selection import train_test_split

/home/rajk/miniconda3/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rajk/miniconda3/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rajk/miniconda3/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rajk/miniconda3/envs/donkey/lib/python3.7/s

In [2]:
SRC_DIR = os.path.dirname(os.path.abspath("."))
sys.path.insert(0, SRC_DIR)

In [3]:
from src.model import DonkeyNet
from src.utils import load_data, blur_img, data_generator, clip_steering

In [4]:
%load_ext line_profiler

# Loading Data

In [5]:
verbose = True
DATA_PATH = r"/home/rajk/Machine_Learning/DonkeyCar/data/env_1/"
img_key, steering_key, throttle_key = "cam/image_array", "user/angle", "user/throttle"

In [6]:
%lprun -f load_data load_data(DATA_PATH, img_key=img_key, steering_key=steering_key, throttle_key=throttle_key, extensions=[".json"], exclude_files=["meta.json"], verbose=verbose)

Number of files excluded: 1
Found 15328 files with one of these extensions ['.json'] in '/home/rajk/Machine_Learning/DonkeyCar/data/env_1/'
 Loading images : ██████████████████████████████████████████████████ 100.00% (15328/15328)



In [6]:
X, Y = load_data(DATA_PATH, img_key=img_key, steering_key=steering_key, throttle_key=throttle_key, 
                 extensions=[".json"], exclude_files=["meta.json"], verbose=verbose)
# Filter out throttle readings
Y = np.reshape(Y[:, 0], (len(Y), 1))

Number of files excluded: 1
Found 15328 files with one of these extensions ['.json'] in '/home/rajk/Machine_Learning/DonkeyCar/data/env_1/'
 Loading images : ██████████████████████████████████████████████████ 100.00% (15328/15328)

# Function load_data took 4.723 seconds!


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=853)

# Model Initialization

In [10]:
version = 1
input_shape = X_train[0].shape
loss_func = tf_v1.losses.mean_squared_error
optimizer = tf_v1.train.AdamOptimizer(learning_rate=1e-4)
model = DonkeyNet(version=version, input_shape=input_shape, loss_func=loss_func, optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model DonkeyNetV1Model with 15 layers built!

Instructions for updating:
Use tf.cast instead.


# Model Training and Testing

In [11]:
SAVE_MODEL_PATH = os.path.join(SRC_DIR, "models", f"DonkeyNetV{version}Model", "model.chkpt")
os.makedirs(os.path.dirname(SAVE_MODEL_PATH), exist_ok=True)
GPU_OPTIONS = tf_v1.GPUOptions(per_process_gpu_memory_fraction=0.75)
TF_CONFIG = tf_v1.ConfigProto(gpu_options=GPU_OPTIONS, allow_soft_placement=True)

In [ ]:
epochs = 20
with tf_v1.Session(config=TF_CONFIG) as sess:
    sess.run(tf_v1.global_variables_initializer())
    train_data_gen = data_generator(X_train, Y_train, epochs=epochs, batch_size=256, preprocessors=[blur_img])
    test_data_gen = data_generator(X_test, Y_test, batch_size=1)
    print("Training the model!")
    train_losses = model.run(sess, data_gen=train_data_gen, training=True)
    print(f"Mean training loss: {np.mean(train_losses):.3f}")
    print("Testing the model")
    test_losses = model.run(sess, data_gen=test_data_gen, training=False)
    model.save_model(sess, SAVE_MODEL_PATH)
    print(f"Mean testing loss: {np.mean(test_losses):.3f}")

# Training and Testing loss plots

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=[20, 5], squeeze=False)
axes[0, 0].set_title("Training losses")
axes[0, 0].plot(train_losses)
axes[0, 1].set_title("Testing losses")
axes[0, 1].plot(test_losses)

# Load Model Parameters

In [ ]:
RESTORE_MODEL_PATH = os.path.join(SRC_DIR, "models", "model.chkpt")
with tf_v1.Session(config=TF_CONFIG) as sess:
    sess.run(tf_v1.global_variables_initializer())
    model.restore_model(sess, RESTORE_MODEL_PATH)
    test_data_gen = data_generator(X_test, Y_test, batch_size=1)
    test_losses = model.run(sess, data_gen=test_data_gen, training=False)
plt.plot(test_losses)